#Use isto se for a primeira vez usando o datalake da basedosdados.org: <br>
   pip install basedosdados
<br><br>
#Use para baixar a base como microdados_violencia.csv: <br>
   import basedosdados as bd <br>
   df = bd.read_table(dataset_id='br_ms_sinan', table_id='microdados_violencia', billing_project_id="violenciasinan", use_bqstorage_api = True) <br>
   df.to_csv(path_or_buf = 'microdados_violencia.csv')

In [ ]:
# importar bibliotecas usadas
import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers.experimental import preprocessing

In [ ]:
print(tf.__version__)
try:
    physical_devices = tf.config.list_physical_devices('GPU') 
    #tf.config.experimental.set_memory_growth(physical_devices[0], True)
    for device in physical_devices:
        tf.config.experimental.set_memory_growth(device, True)
    print(physical_devices)
except:
    print("No GPU")

In [ ]:
# carregar dados em um dataframe pandas
# para usar o arquivo .csv baixe da pasta do sharepoint ou com os comandos disponíveis no topo do notebook e insira o caminho do arquivo no comando abaixo
df = pd.read_csv("C:/Users/rozan/OneDrive - Fundacao Getulio Vargas - FGV/Eletivas/AdvLearning/microdados_violencia.csv")
# excluir colunas inadequadas para a análise
df = df.drop(['houve_qual_outra_violencia_sexual', 'meio_qual_outro', 'ocorreu_qual_outra', 'outro_local_ocorrencia', 'quais_outras_deficiencias_paciente', 'autor_relacao_outros',
'data_encerramento', 'data_notificacao', 'id_categoria_cid10', 'id_municipio_notificacao', 'id_municipio_6_notificacao', 'id_unidade_notificacao', 'id_regional_saude_notificacao',
'id_municipio_ocorrencia', 'id_municipio_6_ocorrencia', 'id_municipio_residencia', 'id_municipio_6_residencia', 'id_regional_saude_residencia'], axis=1)

In [ ]:
# resumo dos dados
with pd.option_context('display.max_rows', 5, 'display.max_columns', None): 
    display(df)

In [ ]:
# separando labels e features
features = df.copy()
labels = features.pop('outras_vezes_ocorrencia')

In [ ]:
inputs = {}

for name, column in features.items():
  dtype = column.dtype
  if dtype == object:
    dtype = tf.string
  else:
    dtype = tf.float32

  inputs[name] = tf.keras.Input(shape=(1,), name=name, dtype=dtype)

# concatenar e normalizar dados numéricos
numeric_inputs = {name:input for name,input in inputs.items()
                  if input.dtype==tf.float32}

x = layers.Concatenate()(list(numeric_inputs.values()))
norm = preprocessing.Normalization()
norm.adapt(np.array(df[numeric_inputs.keys()]))
all_numeric_inputs = norm(x)
preprocessed_inputs = [all_numeric_inputs]

all_numeric_inputs

In [ ]:
# modelo de regressão
# SGD como otimizador
modelo = tf.keras.Sequential([
  layers.Dense(64),
  layers.Dense(1)
])

modelo.compile(loss = tf.losses.MeanSquaredError(),
                      optimizer = tf.optimizers.SGD())

In [ ]:
modelo.fit(features, labels, epochs=10)

In [ ]:
features.dtypes